In [1]:
import pandas as pd
import requests
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
df = pd.read_csv("tracks_per_year.txt",delimiter='<SEP>')
df.to_csv('tracks_per_year.csv')
df.columns

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Index(['YEAR', 'ID1', 'ArtistName', 'SongTitle'], dtype='object')

In [3]:
del df['ID1']

df.head()

,YEAR,ArtistName,SongTitle
0,1922,Alberta Hunter,Don't Pan Me
1,1922,Barrington Levy,Warm And Sunny Day
2,1922,Barrington Levy,Looking My Love
3,1922,Barrington Levy,Warm And Sunny Day
4,1922,Barrington Levy,Mandela You're Free


In [4]:
df.dtypes

YEAR           int64
ArtistName    object
SongTitle     object
dtype: object

In [5]:
# Get names of indexes for which column year is less than 2008
indexNames = df[ df['YEAR'] < 2008 ].index
 
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [6]:
df.head()

,YEAR,ArtistName,SongTitle
440357,2008,$jammie the Money,Getting warm
440358,2008,00Genesis,Dream Catcher
440359,2008,10 Years,Proud of You
440360,2008,10 Years,Alabama
440361,2008,10 Years,Picture Perfect (In Your Eyes)


In [7]:
df.shape

(75219, 3)

In [8]:
#setting up link to spotify api w spotipy
client_id = "5028c957611248149d8c04007258f254"
client_secret = "aa56a507cb944f488f8835062a94115c"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
#next steps = make a loop to iterate through each row of the dataframe
#save each song's id, audio features, and popularity score in a column
#if desired, we can make the dataset bigger

In [10]:
#test dataframe
dft = df.head(1000)
dft

,YEAR,ArtistName,SongTitle
440357,2008,$jammie the Money,Getting warm
440358,2008,00Genesis,Dream Catcher
440359,2008,10 Years,Proud of You
440360,2008,10 Years,Alabama
440361,2008,10 Years,Picture Perfect (In Your Eyes)
...,...,...,...
441352,2008,Alex Gopher,Belmondo (Just A Band remix)
441353,2008,Alex Gopher,Aurora (Dada Life Remix)
441354,2008,Alex Gopher,Aurora (Shinichi Osawa Remix)
441355,2008,Alex Gopher,Belmondo


In [11]:
artistname="Falsalarma"
search_str = "Date Prisa"
result= sp.search(search_str+":"+artistname,type="track,artist")
idt = result["tracks"]["items"][0]["id"]
ft1 = sp.audio_features(idt)
ft2 = pd.DataFrame(ft1)
ft2["popularity"] = sp.track(idt)["popularity"]
ft2["Artist"] = artistname
ft2["Track"] = search_str
ft2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,popularity,Artist,Track
0,0.728,0.628,11,-5.809,0,0.272,0.00774,0,0.172,0.576,...,audio_features,4u9FujDOiwXGOUwrxdROdG,spotify:track:4u9FujDOiwXGOUwrxdROdG,https://api.spotify.com/v1/tracks/4u9FujDOiwXG...,https://api.spotify.com/v1/audio-analysis/4u9F...,271227,4,13,Falsalarma,Date Prisa


In [12]:
result

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Date+Prisa%3AFalsalarma&type=artist&offset=0&limit=10',
  'items': [],
  'limit': 10,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'tracks': {'href': 'https://api.spotify.com/v1/search?query=Date+Prisa%3AFalsalarma&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5vHV7UQFBAZzW0gNb3TaZr'},
       'href': 'https://api.spotify.com/v1/artists/5vHV7UQFBAZzW0gNb3TaZr',
       'id': '5vHV7UQFBAZzW0gNb3TaZr',
       'name': 'Falsalarma',
       'type': 'artist',
       'uri': 'spotify:artist:5vHV7UQFBAZzW0gNb3TaZr'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      '

In [13]:
#result['artists'].keys()
result.keys()

dict_keys(['artists', 'tracks'])

In [14]:
pop = ft2["popularity"]
pop

0    13
Name: popularity, dtype: int64

In [15]:
ft2.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'popularity', 'Artist', 'Track'],
      dtype='object')

In [16]:
ft2 = ft2.drop(columns=['analysis_url', 'id', 'track_href', 'type', 'uri'])

In [17]:
ft2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,Artist,Track
0,0.728,0.628,11,-5.809,0,0.272,0.00774,0,0.172,0.576,92.1,271227,4,13,Falsalarma,Date Prisa


In [18]:
pop = ft2["popularity"].astype(int)
pop

0    13
Name: popularity, dtype: int64

In [19]:

# iterate over the dataframe row by row
for index_label, row in dft.iterrows():
   # For each row update the 'Bonus' value to it's double  

    try: 
        artistname= row['ArtistName']
        search_str =  row['SongTitle']
        print(artistname+ ","+ search_str )
        result= sp.search(search_str+":"+artistname,type="track,artist")
        print(result["tracks"]["items"][0]["id"])
        idr = result["tracks"]["items"][0]["id"]
        ftr = sp.audio_features(idr)
        ftr2 = pd.DataFrame(ftr)
        ftr2["popularity"] = sp.track(idr)["popularity"]
        ftr2["Artist"] = artistname
        ftr2["Track"] = search_str
        ft2 = ft2.append(ftr2)
        
    except: 
        print ('data not found')

$jammie the Money,Getting warm
1boN0BWdCthrH6ZjND3MF6


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


00Genesis,Dream Catcher
1GVAHPHe229qqje2K1R8Ru
10 Years,Proud of You
1JMGklRieQkiGp7FsYx4n7
10 Years,Alabama
5UGBFGK0cmm5pS9zwXl7H4
10 Years,Picture Perfect (In Your Eyes)
0k6LfLD7vowO9ZE38LfPl3
10 Years,So Long_ Good-bye
data not found
10 Years,Beautiful
6AgtIN7FyBd4zJhZI7mrsz
10 Years,All Your Lies
2Perdw5NdmyHJuSOzXYUTp
10 Years,Russian Roulette
41jfrcIFbuFZe2dQANzmb1
10 Years,Actions & Motives
2QgTdBYfwIDRy892unFoRm
10 Years,Dying Youth
1C8I5fruw5Ywm7WjbYdbxv
10 Years,Drug of Choice
5bwJVTyYzj66jfvXoU9L3C
10 Years,11:00am (Daydreamer)
7DiCPnLujrOyFtk2lqxERc
10 Years,Focus
3TRJYRAIoM4LMgqp1z7fEi
10 Years,Just Can't Win
0p0Z2fcPcYKaXNsUKqmzoX
1000 Robota,Wir bauen eine neue Stadt
5SMqDZrlRnWcIWmAE5uvdp
1000 Robota,Lauf lauf lauf
4yUWQuSruUKXtcZmlH8fx3
1000 Robota,Hamburg brennt
6Jzz2zNry8jxhfuNVex9ng
1000 Robota,Es geht nun mal um etwas
0xWFOd9CtTtC9PTQOXcbFs
1000 Robota,Sachen erleben
3ZGwCwZHMzRt8DhCFqQjhP
16 Bit Lolitas,Starfish Prime
data not found
16 Bit Lolitas,Karmageddon
data

0itk5CL8jov8zacXo2R3Hk
36 Crazyfists,Northern November
2U88JSkzyCcEG6CoaMV29E
36 Crazyfists,The Tide And Its Takers
7bQYnNv8VjbpY4iOZHQeq9
36 Crazyfists,Clear The Coast
46I5VKwkpDxAuH7SeGhr0d
3OH!3,HOLLERTILYOUPASSOUT (Explicit Album Version)
data not found
3OH!3,CHOKECHAIN (Explicit Album Version)
data not found
3OH!3,IMNOTYOURBOYFRIENDBABY (Amended Album Version)
data not found
3OH!3,STILLAROUND [BIGMIX] (Bonus Version)
data not found
3OH!3,ICANTDOITALONE (Amended Album Version)
data not found
3OH!3,ICANTDOITALONE (Explicit Album Version)
data not found
3OH!3,IMNOTYOURBOYFRIENDBABY (Explicit Album Version)
data not found
3OH!3,COLORADOSUNRISE (Explicit Album Version)
data not found
3OH!3,PUNKB*TCH (Explicit Album Version)
data not found
3OH!3,CHOKECHAIN (Amended Album Version)
data not found
3OH!3,PHOTOFINNISH (Explicit Album Version)
data not found
3OH!3,DONTTRUSTME [BENNYBLANCOREMIX] FEATURINGKIDCUDI (Explicit Bonus Version)
data not found
3OH!3,TAPP (Album Version)
data not found


6kptnRVmb1llH8eNQMUnOs
A Skylit Drive,Knights Of The Round
0wWIO1TB4QWHxnSIuOOri5
A Skylit Drive,Ex Machina
2bkSpauZI7KL7lBRdd2uc2
A Skylit Drive,Balance
2rzOkuE5oJN1xSUeD9iMnK
A Skylit Drive,All It Takes For Your Dreams To Come True
7DtwEwGYdcZDlXDoDmW0zo
A Skylit Drive,I'm Not A Thief_ I'm A Treasure Hunter
data not found
A Skylit Drive,This Isn't The End
7cDXrlJ9M1WSZUHyzCxd9y
A Skylit Drive,In The Beginning There Was Void
1cHLUJJJsXhwIAr1SFH2eM
A Skylit Drive,Eris And Dysnomia
4T9nIVoqW5Xsngaorl1j1T
A Skylit Drive,Sleepwalker
0fVE3Rr46hzaCjaILt9IJB
A Skylit Drive,My Disease
0Btgh8fj1t0DalPHrNPhrV
A Static Lullaby,Scavenger
6UMnDN7dYiHZGAf2uEqqOp
A Static Lullaby,Mourning Would Come
0lfsoGti319IBTB3OtUVJb
A Static Lullaby,Aller Au Diable
2p89tsBGjLtrsxRC9SiNqz
A Static Lullaby,Rattlesnake!
3L12dlHEFjoaMz2sn2CWK6
A Static Lullaby,Under Water Knife Fight
2tgyURUjx0GdOGv7rsxm0m
A Static Lullaby,The Turn
0N0m7EwI9rM7fgAYGhWyez
A Static Lullaby,Everybody's Got A Lil' Fonz n'em
6l8VSzbBc9

6jtkBsbFr6HEGFu4gOGGtM
Abe Vigoda,Dead City / Waste Wilderness
105g3CuSo6qfUKlaZ8M4OW
Abe Vigoda,Skeleton
105g3CuSo6qfUKlaZ8M4OW
Abe Vigoda,Endless Sleeper
4aO4KXPbzDhIalX8ybeKcz
Abe Vigoda,Hyacinth Grrls
3SIP4bSx1nlJy52O0bjDLq
Abe Vigoda,Whatever Forever
3yiQTs2Ypzh8Ecfw5U1NXB
Abe Vigoda,Live-Long
1ae70b2kpm0dGEfPQans18
Abe Vigoda,Animal Ghosts
3dLWwUHhRROLRPQX8MtUFI
Abe Vigoda,The Garden
43b3XTEmwCdZpmAV6x4gMz
Abe Vigoda,Lantern Lights
20WddDh6fGXLHlLOzviD6H
Abe Vigoda,Gates
1hiYMMCmnB84yngyaGsfjT
Abe Vigoda,Cranes
0iIWF23EE8glXW2b39tigH
Abe Vigoda,Visi Rings
4X1OpLiBFdDXLJAjXAc9Up
Abel Ramos,Jakarta
0igfNs5U4rffORPTFHF0TS
Able Baker Fox,Brand New Moses
7yQzMn1Qbri72rHMpijkLk
Able Baker Fox,Palindramatics
7zIjITbFGOvhHTXJWQQorO
Able Baker Fox,Folding Pocket Blades
1GsDUBiW4wYyecViWGi5UY
Able Baker Fox,Dead Space
074s5tq1P8jEa4a3n2YqP7
Able Baker Fox,Twenty Centuries
5GrVkghus5PB4BK7ywOlc7
Able Baker Fox,Face On Fire
3ARqfKOdWoSw1skJkbL5T5
Able Baker Fox,What Doesn`t Kill You
data not

data not found
Adorned Brood,Drunken Sailor
data not found
Adorned Brood,Am Grunde des Meeres
data not found
Adorned Brood,Storm
data not found
Adrenalin O.D.,Love Song
5qfy7a2XPhISKJNj0vdVEm
Adrenalin O.D.,I'd Rather Be Asleep
69kuceXOdUmLmURnzLK8DQ
Adrian Belew,Looking For A U.F.O.
1wglSbjmiG8fIWKWwD3I9L
Adrian Belew,Neptune Pool
2Lqprpbcz53oleEkTMm5Yf
Adrian Belew,Shoe Salesman
3ZKVEjia14sR53xOLgjiEn
Adriana Calcanhotto,Teu Nome Mais Secreto
1lTCAkCKlsxEWvquZ4SJT9
Adriana Calcanhotto,Porto Alegre (Nos Braços De Calipso)
61yLBmurWJnssq758TJMDN
Adriana Calcanhotto,Seu Pensamento
5E9FgjQiyEckt2KUxb94vv
Adriana Calcanhotto,Para Lá
3v8PLY8ONWd5Bi90N4pApB
Adrienne Pierce,Downside Of Love
7DAPaOnsJqromqT7cgvZ8I
Adrienne Pierce,One Perfect Day
6ywt3lWeGmafS2ifTuaXti
Adrienne Pierce,Faultline
5KKZUueIeBMB96mFgQMYLa
Adrienne Pierce,Under That Cloud
5UqkuJBQrNHISRbUasmUKo
Adrienne Pierce,Walk Through Me
7Fhw8wgORPaC9VSWBQzCae
Adrienne Pierce,One Perfect Day
6ywt3lWeGmafS2ifTuaXti
Adrienne Pier

2W36CxoEbBvk8UdGMwVNeC
Airwave,Punjabi Child
1wJ59wsns6ngOL9yF79kvY
Akesson,Perfect Blue
2NJJTJWyactyScMD5Pxtk7
Akesson,Perfect Blue
2NJJTJWyactyScMD5Pxtk7
Akesson,Perfect Blue
2NJJTJWyactyScMD5Pxtk7
Akissforjersey,Salus Suas Extanderealas Concedit
4GEwjobyrUiOnF09909dUh
Akissforjersey,Leave That Man Be (Acoustic)
10FkNRHAzVmsqTTJvC3vSG
Akissforjersey,The Fire
0vqy1ColnXDLcfXG0oi1Jr
Akissforjersey,Father Part The Clouds In The Sky (Acoustic)
6FN7kTEc4VWUKFTJH7GZsw
Akissforjersey,A Tree And Its Fruit
2lAT5nhC43ERniJjEqNAMm
Akissforjersey,Believe
2YAZmdNVipa6qkw9uohHkF
Akissforjersey,Candi
1RvmloLQLqzOO1WwwSn1vS
Akissforjersey,Cocaine
0X3XoLDIFGMKwF1GUcYXGX
Akissforjersey,Dat Dat
5KohaOTpcv83G6z7Xw92IS
Akissforjersey,Marvin Gaye And How I Stole Myself (Acoustic)
4U2MrFb5OX5Fhemd5VGHLM
Akissforjersey,The Evidence
7vjLqHCuxA1HRBzCxLKKZV
Akissforjersey,Devices
1M3pLyEqBoMD8cRwBnXOCx
Akissforjersey,Faces
07rLTg2TqHwTuqWb7hhat3
Akissforjersey,The Flood
6hD2vD6zDfK7htW9SlPQFq
Akissforjersey,Oh

Alejandro Fernandez,El Reloj
7arKL7omt8YXDKGw3A1PtN
Alejandro Fernandez,Cuando Ya No Me Quieras
5qoViJCj9o7AimDNSXOm5C
Aleks Syntek,Sexo Pudor Y Lágrimas
7ji21BRR5NpJC4rWdsyJAP
Aleks Syntek,Eres Agua
3J9TtbGZb012CFBwvagR4Z
Aleks Syntek,Un Paso Más
7HOzuK0VODg3P4FcTwHgdS
Aleks Syntek,Sexo Pudor Y Lágrimas
7ji21BRR5NpJC4rWdsyJAP
Alesana,All Night Dance Parties In The Underground Palace (album)
data not found
Alesana,Red And Dying Evening (album)
data not found
Alesana,And They Call This Tragedy (album)
data not found
Alesana,This Is Usually The Part Where People Scream (album)
data not found
Alesana,Better Luck Next Time_ Prince Charming (album)
data not found
Alesana,Beautiful In Blue
6cJh50j9mdd0fk5WnW8FQd
Alesana,Obsession Is Such An Ugly Word (album)
data not found
Alesana,Endings Without Stories (album)
data not found
Alesana,Sweetheart_ You Are Sadly Mistaken (album)
data not found
Alesana,Seduction (album)
data not found
Alesana,As You Wish (album)
data not found
Alesana,A Most Pr

In [20]:
ft2 = ft2.drop(columns=['analysis_url', 'id', 'track_href', 'type', 'uri'])


In [35]:
test_dataset = ft2
test_dataset.shape

(781, 16)

In [36]:
test_dataset.head()

,Artist,Track,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,valence
0,Falsalarma,Date Prisa,0.007740,0.728,271227,0.628,0.000000,11,0.1720,-5.809,0,13,0.2720,92.100,4,0.5760
0,$jammie the Money,Getting warm,0.002410,0.673,270000,0.911,0.686000,7,0.0846,-8.082,1,0,0.0744,130.006,4,0.3290
0,00Genesis,Dream Catcher,0.015400,0.772,181107,0.840,0.607000,1,0.1070,-8.262,1,0,0.3110,93.091,4,0.6570
0,10 Years,Proud of You,0.815000,0.319,351587,0.422,0.053700,9,0.2890,-10.800,1,26,0.0292,108.481,3,0.0407
0,10 Years,Alabama,0.000991,0.330,234333,0.863,0.000252,9,0.5090,-3.109,1,24,0.0419,175.897,4,0.1820


In [22]:
test_dataset.to_csv (r'C:\Users\Suhaib Kiani\Desktop\dataset_10k.csv', index = False, header=True)

In [40]:
#test_dataset.popularity.value_counts()


## Audio Analysis Exploration 

In [28]:
ft3 = sp.audio_analysis(idt)
#ft3

In [29]:
#additional features for consideration 
print(ft3['track']['loudness'])
print(ft3['track']['tempo'])

-5.809
92.1


In [30]:
#Audio Analysis Key Map
print("MAIN:", ft3.keys())
print("--------------------")
print("TRACK:", ft3['track'].keys())

MAIN: dict_keys(['meta', 'track', 'bars', 'beats', 'tatums', 'sections', 'segments'])
--------------------
TRACK: dict_keys(['num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])


In [31]:
audio_seg = pd.DataFrame(ft3['segments'])
audio_seg

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,pitches,timbre,loudness_end
0,0.00000,0.08132,0.000,-60.000,0.00000,-60.000,"[0.322, 0.258, 0.213, 0.195, 0.418, 0.425, 0.9...","[1.074, 177.528, 38.228, -55.925, 40.387, -58....",NaN
1,0.08132,0.30181,1.000,-60.000,0.02179,-25.316,"[0.673, 0.797, 0.511, 0.418, 0.356, 0.416, 0.6...","[18.803, 182.762, -31.707, 273.125, 55.172, -2...",NaN
2,0.38313,0.30190,1.000,-60.000,0.04909,-24.740,"[0.528, 1.0, 0.601, 0.275, 0.327, 0.44, 0.454,...","[19.602, 169.856, -14.223, 236.859, 52.107, 50...",NaN
3,0.68503,0.29610,1.000,-59.823,0.02005,-24.544,"[0.683, 0.706, 0.518, 0.454, 0.363, 0.413, 0.7...","[18.98, 183.753, -34.406, 275.226, 54.623, -32...",NaN
4,0.98113,0.30748,1.000,-60.000,0.05406,-24.869,"[0.522, 1.0, 0.601, 0.281, 0.329, 0.449, 0.456...","[18.422, 164.723, -18.56, 222.579, 58.341, 89....",NaN
...,...,...,...,...,...,...,...,...,...
1251,267.99274,0.16286,0.267,-55.171,0.05893,-52.359,"[0.599, 0.704, 0.479, 0.388, 0.513, 0.559, 0.3...","[6.002, 154.518, 87.439, 14.093, 47.091, -23.7...",NaN
1252,268.15560,0.15701,0.159,-55.912,0.05816,-53.492,"[0.555, 0.61, 0.501, 0.454, 0.432, 0.566, 0.36...","[4.962, 154.963, 76.07, 13.667, 47.641, -27.57...",NaN
1253,268.31261,0.16766,0.206,-57.443,0.06426,-54.800,"[0.623, 0.914, 0.673, 0.611, 0.615, 0.624, 0.3...","[3.811, 160.591, 65.91, 0.211, 45.663, -29.065...",NaN
1254,268.48027,0.16898,0.224,-57.780,0.06454,-55.314,"[0.6, 0.805, 0.557, 0.436, 0.69, 0.659, 0.389,...","[3.209, 164.964, 59.097, -6.506, 43.174, -26.8...",NaN


In [32]:
audio_sec = pd.DataFrame(ft3['sections'])
audio_sec

,start,duration,confidence,loudness,tempo,tempo_confidence,key,key_confidence,mode,mode_confidence,time_signature,time_signature_confidence
0,0.00000,5.60193,1.000,-19.171,98.409,0.719,2,0.000,1,0.000,4,1.0
1,5.60193,63.88831,0.883,-5.612,92.161,0.526,11,0.330,0,0.331,4,1.0
2,69.49024,111.55103,0.477,-5.417,91.958,0.536,11,0.457,0,0.397,4,1.0
3,181.04127,44.36226,0.109,-5.135,92.215,0.594,1,0.774,1,0.510,4,1.0
4,225.40353,41.55086,0.215,-6.419,92.317,0.611,11,0.028,0,0.316,4,1.0
5,266.95439,4.27227,0.881,-53.713,0.000,0.000,1,0.000,1,0.000,4,1.0


In [33]:
audio_tats = pd.DataFrame(ft3['tatums'])
#audio_tats

In [34]:
audio_beats = pd.DataFrame(ft3['beats'])
#audio_beats

In [79]:
audio_bars = pd.DataFrame(ft3['bars'])
#audio_bars